# Melbourne Housing Price Prediction - Task 8.2
## 1. Data Acquisition and Database Setup 
This section collects housing data from releastate.com and stores it in a SQLite database for further analysis.

In [6]:
import sqlite3
from pathlib import Path

DATA_DIR = Path("../data")
DB_PATH = DATA_DIR / "melbourne_housing.db"
DATA_DIR.mkdir(exist_ok=True)

with sqlite3.connect(DB_PATH) as conn:
    conn.execute("""
    CREATE TABLE IF NOT EXISTS sales (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        suburb TEXT NOT NULL,
        postcode TEXT,
        address TEXT,
        property_type TEXT,
        bedrooms INTEGER,
        bathrooms INTEGER,
        car_spaces INTEGER,
        land_size_m2 REAL,
        building_size_m2 REAL,
        sold_price INTEGER,
        sold_date TEXT,
        source_url TEXT,
        source TEXT DEFAULT 'realestate.com.au',
        scraped_at TEXT DEFAULT (datetime('now'))
    );
    """)

    conn.execute("""
    CREATE UNIQUE INDEX IF NOT EXISTS idx_unique_listing
    ON sales(suburb, address, sold_date, sold_price);
    """)

print("✅ Database created successfully at:", DB_PATH.resolve())


✅ Database created successfully at: /Users/baoan/Study/University/Deakin/Sem 3 - 2025/SIT720 - ML/Week tasks/Week8/8.2/Melbourne-housing-regression/data/melbourne_housing.db


# Create CSV Template

In [8]:
import pandas as pd 

#CSV template
template_cols = [
    "suburb", "postcode", "address", "property_type",
    "bedrooms", "bathrooms", "car_spaces",
    "land_size_m2", "building_size_m2",
    "sold_price", "sold_date",
    "source_url"
]

template_path = DATA_DIR / "sales_template.csv"
pd.DataFrame(columns=template_cols).to_csv(template_path, index=False)

print(f"Template CSV file created at {template_path}")

Template CSV file created at ../data/sales_template.csv


# Scarping data and save into the csv file

In [14]:
import time
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import csv

Filled_CSV = DATA_DIR / "sales_filled.csv"

POSTCODE_MAP = {
    "Melbourne": "3000",
    "Toorak": "3142",
    "Camberwell": "3124",
}

SUBURB_URLS = {
    "Melbourne": "https://www.realestate.com.au/sold/in-melbourne,+vic+3000/list-{}",
    "Toorak": "https://www.realestate.com.au/sold/in-toorak,+vic+3142/list-{}",
    "Camberwell": "https://www.realestate.com.au/sold/in-camberwell,+vic+3124/list-{}",
}

TARGET_PER_SUBURB = 60
MAX_PAGES = 10
SLEEP_SECONDS = 2.0

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)",
    "Accept-Language": "en-AU,en;q=0.9",
}

# Helper Functions
def fetch_html(url: str) -> str:
    r = requests.get(url, headers = HEADERS, timeout = 20)
    r.raise_for_status()
    return r.text

def extract_property_links(html: str) -> list[str]:
    soup = BeautifulSoup(html, "lxml")
    links = set()

    for a in soup.select("a[href]"):
        href = a.get("href", " ")
        if "/property-" in href or "/sold/" in href:
            if href.startswith("/"):
                href = "https://www.realestate.com.au" + href
            if href.startswith("https://www.realestate.com.au/"):
                links.add(href.split("?")[0])

    return sorted(links)

# Ensure sales_filled.csv exists with header
cols = [
    "suburb", "postcode", "address", "property_type",
    "bedrooms", "bathrooms", "car_spaces",
    "land_size_m2", "building_size_m2",
    "sold_price", "sold_date",
    "source_url"
]

if not Filled_CSV.exists():
    with open(Filled_CSV, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(cols)

print("✅ sales_filled.csv ready:", Filled_CSV.resolve())

results = {k: [] for k in SUBURB_URLS.keys()}
seen_urls = set()

for suburb, url_tpl in SUBURB_URLS.items():
    print(f"\n=== Collecting for {suburb} ===")

    for page in range(1, MAX_PAGES + 1):
        if len(results[suburb]) >= TARGET_PER_SUBURB:
            break
    
        url = url_tpl.format(page)
        print(f"🔍 Fetching {url}...")

        try: 
            html = fetch_html(url)
            urls = extract_property_links(html)

            new_urls = []
            for u in urls:
                if u not in seen_urls:
                    seen_urls.add(u)
                    new_urls.append(u)
            
            for u in new_urls:
                if len(results[suburb]) >= TARGET_PER_SUBURB:
                    break
                results[suburb].append(u)

            print(f"  found={len(urls)} | new={len(new_urls)} | collected={len(results[suburb])}")
            time.sleep(SLEEP_SECONDS)

        except Exception as e:
            print("  ❌ error:", repr(e))
            time.sleep(SLEEP_SECONDS)


✅ sales_filled.csv ready: /Users/baoan/Study/University/Deakin/Sem 3 - 2025/SIT720 - ML/Week tasks/Week8/8.2/Melbourne-housing-regression/data/sales_filled.csv

=== Collecting for Melbourne ===
🔍 Fetching https://www.realestate.com.au/sold/in-melbourne,+vic+3000/list-1...
  ❌ error: HTTPError('429 Client Error: Too Many Requests for url: https://www.realestate.com.au/sold/in-melbourne,+vic+3000/list-1')
🔍 Fetching https://www.realestate.com.au/sold/in-melbourne,+vic+3000/list-2...
  ❌ error: HTTPError('429 Client Error: Too Many Requests for url: https://www.realestate.com.au/sold/in-melbourne,+vic+3000/list-2')
🔍 Fetching https://www.realestate.com.au/sold/in-melbourne,+vic+3000/list-3...
  ❌ error: HTTPError('429 Client Error: Too Many Requests for url: https://www.realestate.com.au/sold/in-melbourne,+vic+3000/list-3')
🔍 Fetching https://www.realestate.com.au/sold/in-melbourne,+vic+3000/list-4...
  ❌ error: HTTPError('429 Client Error: Too Many Requests for url: https://www.realestat

KeyboardInterrupt: 

## Write the collected rows into csv

In [ ]:
rows_written = 0

with open(Filled_CSV, "a", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=cols)

    for suburb, urls in results.items():
        for u in urls:
            writer.writerow({
                "suburb": suburb,
                "postcode": POSTCODE_MAP.get(suburb),
                "address": "",
                "property_type": "",
                "bedrooms": "",
                "bathrooms": "",
                "car_spaces": "",
                "land_size_m2": "",
                "building_size_m2": "",
                "sold_price": "",
                "sold_date": "",
                "source_url": u
            })
            rows_written += 1

print("\n✅ Scraping complete")
for suburb in results:
    print(f"{suburb}: {len(results[suburb])} properties")
print("Total rows written:", rows_written)
